# [1] 基于内容的推荐系统（Content-Based Filtering）

**学习顺序：第1个** - 这是推荐系统入门的最佳起点

基于内容的推荐系统通过分析物品的特征（如电影的类型、演员、导演等）来推荐与用户历史偏好相似的物品。

## 为什么先学这个？
- ✅ 最直观易懂，不需要其他用户数据
- ✅ 可以处理冷启动问题
- ✅ 为后续方法打下基础

## 核心思想
- 如果用户喜欢某个物品，那么具有相似特征的其他物品也应该被推荐
- 不需要其他用户的数据，可以处理冷启动问题
- 推荐结果具有可解释性

In [1]:
# 导入必要的库
import sys
import os

# 确保导入项目本地的 matrix_factorization 模块，而不是 conda 环境中安装的包
# 将项目根目录添加到 Python 路径的最前面
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matrix_factorization import ContentBasedRecommender

In [2]:
# 加载数据
cols = ['user_id', 'item_id', 'rating', 'timestamp']
movie_data = pd.read_csv('../data/ml-100k/u.data', names=cols, sep='\t', usecols=[0, 1, 2], engine='python')

# 加载电影特征数据（类型信息）
item_cols = ['item_id'] + [f'genre_{i}' for i in range(19)]
item_data = pd.read_csv('../data/ml-100k/u.item', sep='|', names=item_cols, encoding='latin-1', usecols=[0] + list(range(5, 24)))

print(f"评分数据形状: {movie_data.shape}")
print(f"物品特征数据形状: {item_data.shape}")
item_data.head()

评分数据形状: (100000, 3)
物品特征数据形状: (1682, 20)


,item_id,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [3]:
# 准备数据
X = movie_data[['user_id', 'item_id']]
y = movie_data['rating']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"训练集大小: {len(X_train)}")
print(f"测试集大小: {len(X_test)}")

训练集大小: 80000
测试集大小: 20000


In [4]:
# 训练基于内容的推荐模型
content_model = ContentBasedRecommender(min_rating=1, max_rating=5, verbose=1)
content_model.fit(X_train, y_train, item_features=item_data)

print("\n模型训练完成！")


模型训练完成！


In [5]:
# 预测测试集评分
pred = content_model.predict(X_test)

# 计算RMSE
mse = mean_squared_error(y_test, pred)
rmse = mse ** 0.5

print(f"\n测试集 RMSE: {rmse:.4f}")

/Users/boris/Documents/Recommendation System/week3/matrix_factorization/recommender_base.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[517. 686. 321. ... 629. 604. 221.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[:, "item_id"] = X["item_id"].map(self.item_id_map)



测试集 RMSE: 2.7628


In [6]:
# 为特定用户生成推荐
user = 200
items_known = X_train.query('user_id == @user')['item_id']
recommendations = content_model.recommend(user=user, items_known=items_known, amount=10)

print(f"\n为用户 {user} 的推荐:")
print(recommendations)


为用户 200 的推荐:
     user_id  item_id  rating_pred
0        200      290          1.0
983      200     1113          1.0
992      200     1459          1.0
991      200     1558          1.0
990      200     1079          1.0
989      200     1319          1.0
988      200      893          1.0
987      200      438          1.0
986      200      994          1.0
985      200      691          1.0
